### The effective capacity of neural networks is large enough for a brute-force memorization of the entire data set

### Even optimization on random labels remains easy

-  FITTING RANDOM LABELS AND PIXELS
    - True labels: the original dataset without modification.
    - Partially corrupted labels: independently with probability p, the label of each image is corrupted as a uniform random class.
    - Random labels: all the labels are replaced with random ones.
    - Shuffled pixels: a random permutation of the pixels is chosen and then the same permutation is applied to all the images in both training and test set.
    - Random pixels: a different random permutation is applied to each image independently.
    - Gaussian: A Gaussian distribution (with matching mean and variance to the original image dataset) is used to generate random pixels for each image.

In [ ]:
import tensorflow as tf
import numpy as np
from keras.datasets import cifar10

In [ ]:
image_size = 32
image_channel = 3
learning_rate = 0.001
batch_size = 200
epochs = 600
class_num = 10
# data_augmentation = False

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = np.reshape(x_train, [-1, image_size**2 * image_channel])
num_train = x_train.shape[0]

In [ ]:
def onehot(label):
    onehot_label = []
    for l in label:
        lt = [0]*class_num
        lt[l[0]-1] = 1
        onehot_label.append(lt)
    return np.array(onehot_label)

def partially_corrupted_label(label, probability):
    pcl = []
    for l in label:
        if np.random.random_sample() < probability:
            pcl.append([np.random.randint(class_num)])
        else:
            pcl.append([l[0]])
    return pcl

def random_label(label):
    return np.reshape(np.random.randint(class_num, size=len(label)), [-1, 1])

def label_similarity(label1, label2):
    counter = 0
    for l1, l2 in zip(label1, label2):
        if l1 != l2:
            counter += 1
    return (len(label1) - float(counter)) / len(label1)

def shuffled_pixel(images):
    shuffled_images = []
    permutation = np.random.permutation(image_size**2 * image_channel)
    for image in images:
        shuffled_image = []
        for p in permutation:
            shuffled_image.append(image[p])
        shuffled_images.append(shuffled_image)
    return np.array(shuffled_images)

def guassian(images):
    pixel_num = len(images) * (image_size**2 * image_channel)
    pixel = np.reshape(images, [pixel_num])
    mean = np.ndarray.sum(pixel) / pixel_num
    variance_sum = 0
    for p in pixel:
        variance_sum += (p - mean)**2
    standard_deviation = np.sqrt(variance_sum / pixel_num)
    return np.array(np.random.normal(loc=mean, scale=standard_deviation, 
                                     size=[len(images), (image_size**2 * image_channel)]))

def random_pixel(images):
    random_pixel_images = []
    for image in images:
        random_pixel_image = []
        np.random.shuffle(image)
        random_pixel_images.append(image)
    return np.array(random_pixel_images)

In [ ]:
n_input_size = image_size**2 * image_channel
n_hidden_1 = 512
n_hidden_2 = 512
n_hidden_3 = 512
n_output_size = class_num
sigma = 0.01

weights = {
    'W1': tf.Variable(tf.random_normal([n_input_size, n_hidden_1], stddev=sigma)),
    'W2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2], stddev=sigma)),
    'W3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3], stddev=sigma)),
    'out': tf.Variable(tf.random_normal([n_hidden_3, n_output_size], stddev=sigma))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'b3': tf.Variable(tf.random_normal([n_hidden_3])),
    'out': tf.Variable(tf.random_normal([n_output_size])),

}

def multilayer_perceptron(x, weights, biases):
    layer1 = tf.nn.relu(tf.add(tf.matmul(x, weights['W1']), biases['b1']))
    layer2 = tf.nn.relu(tf.add(tf.matmul(layer1, weights['W2']), biases['b2']))
    layer3 = tf.nn.relu(tf.add(tf.matmul(layer2, weights['W3']), biases['b3']))
    output_layer = tf.nn.relu(tf.add(tf.matmul(layer3, weights['out']), biases['out']))
    return output_layer

In [ ]:
def train(X_train, Y_train):
    x = tf.placeholder(tf.float32, [None, n_input_size], name='input')
    y = tf.placeholder(tf.float32, [None, n_output_size], name='output')
    pred = multilayer_perceptron(x, weights, biases)
    #loss
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
    #optimizer
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)
    #optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))    
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    Y_train = onehot(Y_train)

    with tf.Session() as sess:

        sess.run(tf.global_variables_initializer())
        myIdx =  np.random.permutation(num_train)
        for epoch in xrange(epochs):
            num_batch = num_train / batch_size
            for i in xrange(num_batch):
                x_batch = X_train[myIdx[i*batch_size:(i+1)*batch_size],:]
                y_batch = Y_train[myIdx[i*batch_size:(i+1)*batch_size],:]
                sess.run(optimizer, feed_dict={x: X_train, y: Y_train})
            if (epoch+1) % 10 == 0:
                loss_temp = sess.run(loss, feed_dict={x: X_train, y: Y_train}) 
                accuracy_temp = accuracy.eval({x: X_train, y: Y_train})
                print "(epoch {})".format(epoch+1) 
                print "[Loss / Tranining Accuracy] {:05.4f} / {:05.4f}".format(loss_temp, accuracy_temp)
                print " "

In [ ]:
train(x_train, y_train) #Normal

In [ ]:
train(x_train, partially_corrupted_label(y_train, 0.3)) #Partially Corrupted label experiment

In [ ]:
train(x_train, random_label(y_train)) #Random label experiment

In [ ]:
train(shuffled_pixel(x_train), y_train) #Shuffled pixels experiment

In [ ]:
train(random_pixel(x_train), y_train) #Random pixels experiment

In [ ]:
train(guassian(x_train), y_train) #Guassian generated pixels expermient